<h2>Data Analysis - Batch Processing - Quantification of cell populations</h2>

The following notebook is able to process the .csv files resulting from Batch Processing (Average Intensity or Colocalization) and:

1. Define cell populations based on single or multiple markers (positive, negative or a combination of both)
2. Plot resulting data using Plotly.
3. Extract numbers of cells positive for a marker based on colocalization (using a user-defined threshold).
4. Save results in .csv file (find a name for this).

In [1]:
from pathlib import Path
import pandas as pd
# from utils_data_analysis

In [15]:
# Define the path containing your results
results_path = Path("./results/test_data/3D/MEC0.1")

# Input the method used to define cells as positive for a marker ("avg_int", "coloc") #TODO: "pixel_class"
method = "avg_int"

# Define the channels you want to analyze using the following structure:
# markers = [(channel_name, channel_nr, cellular_location),(..., ..., ...)]
markers = [("ki67", 0, "nucleus"), ("neun", 1, "nucleus"), ("calbindin", 2, "cytoplasm")]

# Define the min_max average intensity parameters to select your populations of interest
# You have the possibility to define populations for the same marker (i.e. neun high and neun low)
# max_values are set to 255 since the test input images are 8-bit, higher bit depths can result in higher max avg_int values
min_max_per_marker = [
    {"marker": "ki67", "min_max": (110,255), "population":"ki67"},
    {"marker": "neun", "min_max": (20,80), "population":"neun_low"},
    {"marker": "neun", "min_max": (80,255), "population":"neun_high"},
    {"marker": "calbindin", "min_max": (10,255), "population":"calbindin"},]

# Define cell populations based on multiple markers (i.e. double marker positive (True) or marker positive (True) and marker2 negative (False))
# Based on populations in min_max_per_marker in case multiple pops per marker are defined, as in the case of "neun"
cell_populations = [
    {"cell_pop": "neun_high", "populations": [("neun_high", True)]},
    {"cell_pop": "neun_low", "populations": [("neun_low", True)]},
    {"cell_pop": "non_prolif", "populations": [("ki67", False)]},
    {"cell_pop": "prolif_neun_high", "populations": [("neun_high", True), ("ki67", True)]},
    {"cell_pop": "prolif_neun_low", "populations": [("neun_low", True), ("ki67", True)]},
    {"cell_pop": "non_prolif_neun_high", "populations": [("neun_high", True), ("ki67", False)]},
    {"cell_pop": "non_prolif_neun_low", "populations": [("neun_low", True), ("ki67", False)]},
    {"cell_pop": "neun_high_+_calbindin_+", "populations": [("neun_high", True), ("calbindin", True)]},
    {"cell_pop": "neun_low_+_calbindin_+", "populations": [("neun_low", True), ("calbindin", True)]},]

In [22]:
# Extract model_name and segmentation type from the results path

model_name = results_path.parts[-1]
segmentation_type = results_path.parts[-2]

# Extract a list of filenames from the results path
per_label_csvs = []

for file_path in results_path.glob("*.csv"):
    if method in str(file_path) and "BP" not in str(file_path) and "SP" not in str(file_path):
        per_label_csvs.append(file_path)

per_label_csvs

[WindowsPath('results/test_data/3D/MEC0.1/HI 1  Contralateral Mouse 8  slide 6 Neun Red Calb Green KI67 Magenta 40x technical replica 1_per_label_avg_int.csv'),
 WindowsPath('results/test_data/3D/MEC0.1/HI 1  Ipsilateral Mouse 8  slide 6 Neun Red Calb Green KI67 Magenta 40x technical replica 1_per_label_avg_int.csv')]

In [27]:
df = pd.read_csv(per_label_csvs[0], index_col=0)

# Extract ROIs
rois = list(df["ROI"].unique())

['CA', 'DG']

In [28]:
df

,filename,ROI,label,nucleus_ki67_avg_int,nucleus_neun_avg_int,cytoplasm_calbindin_avg_int
0,HI 1 Contralateral Mouse 8 slide 6 Neun Red ...,CA,1,90.665614,191.079125,21.777857
1,HI 1 Contralateral Mouse 8 slide 6 Neun Red ...,CA,2,67.898912,88.685856,10.829655
2,HI 1 Contralateral Mouse 8 slide 6 Neun Red ...,CA,3,83.431005,151.437089,20.449377
3,HI 1 Contralateral Mouse 8 slide 6 Neun Red ...,CA,4,72.215565,136.073774,18.086357
4,HI 1 Contralateral Mouse 8 slide 6 Neun Red ...,CA,5,89.546484,149.361235,20.512121
...,...,...,...,...,...,...
3535,HI 1 Contralateral Mouse 8 slide 6 Neun Red ...,DG,2100,68.080780,38.710306,4.465306
3536,HI 1 Contralateral Mouse 8 slide 6 Neun Red ...,DG,2101,77.199660,28.836075,5.104869
3537,HI 1 Contralateral Mouse 8 slide 6 Neun Red ...,DG,2102,55.474522,34.073248,3.339535
3538,HI 1 Contralateral Mouse 8 slide 6 Neun Red ...,DG,2103,59.618042,39.621881,4.599237


In [ ]:
for roi_name in rois:

    